# Import Pathway data in BioPAX 3 format to Node and Relationship files for Knowledge Graph Import

[BioPAX – Biological Pathways Exchange
Language
Level 3](
http://www.biopax.org/release/biopax-level3-documentation.pdf)

[PyBioPax documentation](https://pybiopax.readthedocs.io/en/latest/)
[pybiopax repo](https://github.com/indralab/pybiopax)

Author: Peter W. Rose (pwrose@ucsd.edu)

In [1]:
import os
import pandas as pd
import pybiopax

In [2]:
pd.options.display.max_rows = None  # display all rows
pd.options.display.max_columns = None  # display all columsns
pd.set_option('display.max_colwidth', None) # don't truncate cells

In [3]:
def get_id(obj, database, prefix):
    """Return id of Reactome object"""
    if obj:
        for xref in obj.xref:
            if xref.db == database:
                if ':' in xref.id:
                    # remove CURIE
                    return prefix + xref.id.split(':')[1]
                else:
                    return prefix + xref.id
                
    return ''

In [4]:
def get_version(obj):
    """Return version of Reactome object"""
    for xref in obj.xref:
        if xref.db.startswith('Reactome Database ID Release'):
            return xref.db.split(' ')[4]

    return ''

In [5]:
def get_component_entity_id(component):
    """Return entity id and database name of a component"""
    entity_type = component.__class__.__name__
    entity_id = ''  
    
    if entity_type == 'Complex':
        entity_id = get_id(component, 'Reactome', 'reactome:')
    elif entity_type == 'Protein':
        entity_id = get_id(component.entity_reference, 'UniProt', 'uniprot:')
    elif entity_type  == 'SmallMolecule':
        entity_id = get_id(component.entity_reference, 'ChEBI', 'chebi:')
            
    return entity_id

In [6]:
def create_synonym_string(synonyms):
    if isinstance(synonyms, list):
        synonyms = '|'.join(synonyms)
    
    return synonyms

In [7]:
def extract_pathways(model, node_data, rel_data):
    relationships = {'Pathway': 'IS_A', 'BiochemicalReaction': 'PART_OF'}
        
    for pathway in model.get_objects_by_type(pybiopax.biopax.Pathway):
        row_n = {'entity': 'Pathway', 
                 'id': get_id(pathway, 'Reactome', 'reactome:'), 
                 'reactome_id': get_id(pathway, 'Reactome', ''),
                 'name': pathway.display_name, 
                 'synonyms': create_synonym_string(pathway.name),
                 'version': get_version(pathway)}
        node_data.append(row_n)
    
        for component in pathway.pathway_component:
            entity_type = component.__class__.__name__
            rel = relationships.get(entity_type, '')
            
            if rel != '':
                row_r = {'entity': component.__class__.__name__, 
                         'id': get_id(component, 'Reactome', 'reactome:'),
                         'reactome_id': get_id(component, 'Reactome', ''),
                         'parent_entity': 'Pathway',
                         'parent_id': row_n.get('id'), 
                         'version': get_version(component),
                         'relationship': rel}
                rel_data.append(row_r)
                
    return node_data, rel_data

In [8]:
def extract_reaction(model, node_data, rel_data):
    for reaction in model.get_objects_by_type(pybiopax.biopax.BiochemicalReaction):
        row_n = {'entity': 'Reaction', 
                 'id': get_id(reaction, 'Reactome', 'reactome:'), 
                 'reactome_id': get_id(reaction, 'Reactome', ''),
                 'name': reaction.display_name, 
                 'synonyms': create_synonym_string(reaction.name),
                 'version': get_version(reaction)}
        node_data.append(row_n)
        
        for reaction_entity in [reaction.left, reaction.right]:
#             if reaction_entity == reaction.left:
#                 relationship = 'CONSUMED_BY'
#             else:
#                 relationship = 'PRODUCED_BY'
                
            # simplified version: only one relationship
            relationship = 'PARTICIPATES_IN'
            
            for component in reaction_entity:
                entity_id = get_component_entity_id(component)

                if entity_id != '':
                    row_r = {'entity': component.__class__.__name__, 
                             'id': entity_id,
                             'reactome_id': get_id(component, 'Reactome', ''),
                             'parent_entity': 'BiochemicalReaction',
                             'parent_id': row_n.get('id'), 
                             'version': get_version(component),
                             'relationship': relationship}
                    rel_data.append(row_r)
                
        
    return node_data, rel_data  

In [9]:
def extract_complexes(model, node_data, rel_data):    
    for cmplx in model.get_objects_by_type(pybiopax.biopax.Complex):
        row_n = {'entity': 'Complex', 
                 'id': get_id(cmplx, 'Reactome', 'reactome:'), 
                 'reactome_id': get_id(cmplx, 'Reactome', ''),
                 'name': cmplx.display_name, 
                 'synonyms': create_synonym_string(cmplx.name),
                 'version': get_version(cmplx)}
        node_data.append(row_n)
    
        for component in cmplx.component:         
            entity_id = get_component_entity_id(component)

            if entity_id != '':
                row_r = {'entity': component.__class__.__name__, 
                         'id': entity_id,
                         'reactome_id': get_id(component, 'Reactome', ''),
                         'parent_entity': 'Complex',
                         'parent_id': row_n.get('id'), 
                         'version': get_version(component),
                         'relationship': 'PART_OF'}
                rel_data.append(row_r)
                
    return node_data, rel_data

In [10]:
def extract_proteins(model, node_data):
    for protein in model.get_objects_by_type(pybiopax.biopax.Protein):
        row_n = {'entity': 'Protein', 
                 'id': get_id(protein.entity_reference, 'UniProt', 'uniprot:'), 
                 'reactome_id': get_id(protein, 'Reactome', ''),
                 'name': protein.display_name, 
                 'synonyms': create_synonym_string(protein.name),
                 'version': get_version(protein)}
        node_data.append(row_n)
      
    return node_data

In [11]:
def extract_small_molecules(model, node_data):
    for molecule in model.get_objects_by_type(pybiopax.biopax.SmallMolecule):     
        row_n = {'entity': 'SmallMolecule', 
                 'id': get_id(molecule.entity_reference, 'ChEBI', 'chebi:'), 
                 'reactome_id': get_id(molecule, 'Reactome', ''),
                 'name': molecule.display_name, 
                 'synonyms': create_synonym_string(molecule.name),
                 'version': get_version(molecule)}
        node_data.append(row_n)
      
    return node_data

## Extract node and relationship data

In [12]:
node_data = []
rel_data = []

pathways = ['R-HSA-162582', 'R-HSA-5663202']

for pathway in pathways:
    model = pybiopax.model_from_reactome(pathway)
    node_data, rel_data = extract_pathways(model, node_data, rel_data)
    node_data, rel_data = extract_reaction(model, node_data, rel_data)
    node_data, rel_data = extract_complexes(model, node_data, rel_data)
    node_data = extract_proteins(model, node_data)
    node_data = extract_small_molecules(model, node_data)

Processing OWL elements:   0%|          | 0.00/78.2k [00:00<?, ?it/s]

Processing OWL elements:   0%|          | 0.00/33.3k [00:00<?, ?it/s]

In [13]:
nodes = pd.DataFrame(node_data)
nodes.tail(500)

,entity,id,reactome_id,name,synonyms,version
15222,Protein,uniprot:P36888,R-HSA-9695714,STK-1,STK-1|FLT3 I836del,79
15223,Protein,uniprot:P36888,R-HSA-9695721,STK-1,STK-1|FLT3 N676K,79
15224,Protein,uniprot:P36888,R-HSA-9695722,STK-1,STK-1|FLT3 F691L,79
15225,Protein,uniprot:P36888,R-HSA-9695718,STK-1,STK-1|FLT3 D839G,79
15226,Protein,uniprot:P36888,R-HSA-9699435,FLT3 Y842C,FLT3 Y842C,79
15227,Protein,uniprot:P36888,R-HSA-9682301,STK-1,STK-1|FLT3 D835G,79
15228,Protein,uniprot:P36888,R-HSA-9682313,STK-1,STK-1|FLT3 D835F,79
15229,Protein,uniprot:P36888,R-HSA-9691906,STK-1,STK-1|FLT3 K633N,79
15230,Protein,uniprot:P36888,R-HSA-9691898,STK-1,STK-1|FLT3 K633R,79
15231,Protein,uniprot:P36888,R-HSA-9691900,STK-1,STK-1|FLT3 N841H,79


In [14]:
relationships = pd.DataFrame(rel_data)
relationships.fillna('', inplace=True)
relationships.tail(5)

,entity,id,reactome_id,parent_entity,parent_id,version,relationship
18412,Protein,uniprot:Q9UM73,R-HSA-9701428,Complex,reactome:R-HSA-9701834,79,PART_OF
18413,Protein,uniprot:Q9UM73,R-HSA-9701838,Complex,reactome:R-HSA-9701827,79,PART_OF
18414,Protein,uniprot:Q9UM73,R-HSA-9701418,Complex,reactome:R-HSA-9701837,79,PART_OF
18415,Protein,uniprot:Q9UM73,R-HSA-9699831,Complex,reactome:R-HSA-9700557,79,PART_OF
18416,Protein,uniprot:Q9UM73,R-HSA-9699802,Complex,reactome:R-HSA-9700595,79,PART_OF


In [15]:
def create_nodes(df, entity):
    # create node dataframe
    df_entity = df[(df['entity'] == entity)]
    df_nodes = df_entity[['id', 'name', 'synonyms']].copy()
    df_nodes.drop_duplicates(subset='id', inplace=True)
    df_nodes = df_nodes[(df_nodes['id'] != '')]
    
    return df_nodes

In [16]:
def create_relationships(df, entity, relationship, parent_entity):
    # create node dataframe
    df_entity = df[(df['entity'] == entity) & (df['parent_entity'] == parent_entity) &
                   (df['relationship'] == relationship)]
    
    # create relationship dataframe
    df_relationships = df_entity[['id', 'parent_id']].copy()
    df_relationships.rename(columns={'id': 'from', 'parent_id': 'to'}, inplace=True)
    df_relationships.drop_duplicates(subset=['from', 'to'], inplace=True)
    df_relationships = df_relationships[(df_relationships['from'] != '') & (df_relationships['to'] != '')]
    
    return df_relationships

## Setup Output Directories

In [17]:
NODE_DATA = os.getenv('NODE_DATA', default='../data/nodes/')
RELATIONSHIP_DATA = os.getenv('RELATIONSHIP_DATA', default='../data/relationships/') 

In [18]:
os.makedirs(os.path.join(NODE_DATA), exist_ok=True)
os.makedirs(os.path.join(RELATIONSHIP_DATA), exist_ok=True)

In [19]:
def save_nodes(key, df):
    nodes = create_nodes(df, key)
    nodes.to_csv(os.path.join(NODE_DATA, f'{key}.csv'), index=False)
    print(key, nodes.shape[0])

In [20]:
def save_relationships(key, df):
    relationships = create_relationships(df, key[0], key[1], key[2])
    relationships.to_csv(os.path.join(RELATIONSHIP_DATA, f'{key[0]}-{key[1]}-{key[2]}.csv'), index=False)
    print(f'{key[0]}-{key[1]}-{key[2]}', relationships.shape[0])

In [21]:
node_groups = nodes.groupby('entity')

print('Saving:')
for key, group in node_groups:
    save_nodes(key, group)

Saving:
Complex 4340
Pathway 574
Protein 2554
Reaction 2931
SmallMolecule 246


In [22]:
relationship_groups = relationships.groupby(['entity','relationship', 'parent_entity'])

print('Saving:')
for key, group in relationship_groups:
        save_relationships(key, group)

Saving:
BiochemicalReaction-PART_OF-Pathway 3028
Complex-PARTICIPATES_IN-BiochemicalReaction 4583
Complex-PART_OF-Complex 2327
Pathway-IS_A-Pathway 578
Protein-PARTICIPATES_IN-BiochemicalReaction 1434
Protein-PART_OF-Complex 3549
SmallMolecule-PARTICIPATES_IN-BiochemicalReaction 1897
SmallMolecule-PART_OF-Complex 398
